In [1]:
import numpy as np
import pandas as pd
import sklearn.metrics as m
from sklearn.tree import DecisionTreeRegressor

In [2]:
def make_lags(df, cols, lags):
    return pd.concat([df[col].shift(lag).rename(f"{col}_L{lag}") for col in cols for lag in lags], axis=1)

In [4]:
df = pd.read_csv("../../../data/cleaned/wind_hourly_cleaned.csv")
df = df.sort_values('datetime').reset_index(drop=True)
df['wind_speed'] = df['wind_speed'].astype(float)

In [5]:
LAGS = [1,2,3,23,24,25,48]
df = df.join(make_lags(df[['wind_speed']], ['wind_speed'], LAGS)).dropna()

In [6]:
df.head()

,datetime,wind_speed,wind_speed_L1,wind_speed_L2,wind_speed_L3,wind_speed_L23,wind_speed_L24,wind_speed_L25,wind_speed_L48
48,2024-01-03 00:00:00,0.100000,0.800000,1.000000,0.216667,0.933333,1.050000,0.716667,0.100000
49,2024-01-03 01:00:00,0.616667,0.100000,0.800000,1.000000,0.700000,0.933333,1.050000,0.283333
50,2024-01-03 02:00:00,0.500000,0.616667,0.100000,0.800000,1.450000,0.700000,0.933333,0.200000
51,2024-01-03 03:00:00,0.033333,0.500000,0.616667,0.100000,1.750000,1.450000,0.700000,0.500000
52,2024-01-03 04:00:00,0.116667,0.033333,0.500000,0.616667,0.083333,1.750000,1.450000,0.966667


In [7]:
train_end = len(df) - 720
test_df   = df.iloc[train_end:].copy()


In [8]:
preds = []
for t in range(train_end, len(df)):
    X = df.iloc[t-168:t][[c for c in df.columns if '_L' in c]].values
    y = df.iloc[t-168:t]['wind_speed'].values
    model = DecisionTreeRegressor(max_depth=8, min_samples_leaf=10, random_state=42)
    model.fit(X, y)
    x_next = df.iloc[t][[c for c in df.columns if '_L' in c]].values.reshape(1,-1)
    preds.append(model.predict(x_next)[0])

test_df['pred_dt'] = preds
mae  = m.mean_absolute_error(test_df['wind_speed'], test_df['pred_dt'])
rmse = m.mean_squared_error(test_df['wind_speed'], test_df['pred_dt'])
mape = np.mean(np.abs((test_df['wind_speed'] - test_df['pred_dt'])/test_df['wind_speed']))*100

In [9]:
# --- REPORT PRINTING ---
print("\n=======================================================")
print("  DECISION TREE FORECAST PERFORMANCE REPORT")
print("=======================================================")
print(f"Total Test Observations: {len(test_df):,}")
print("-" * 55)
print("Metric                         Value")
print("-" * 55)
print(f"Mean Absolute Error (MAE):     {mae:20.4f}")
print(f"Root Mean Squared Error (RMSE):{rmse:20.4f}")
print(f"Mean Abs. Percentage Error (MAPE): {mape:18.2f} %")
print("=======================================================")


  DECISION TREE FORECAST PERFORMANCE REPORT
Total Test Observations: 720
-------------------------------------------------------
Metric                         Value
-------------------------------------------------------
Mean Absolute Error (MAE):                   0.8246
Root Mean Squared Error (RMSE):              1.3596
Mean Abs. Percentage Error (MAPE):                inf %


# Step-2 retuned Decision-Tree with cyclical hour-of-day

In [12]:
# ensure the column is datetime *type*
df['date'] = pd.to_datetime(df['datetime'])   # or df['datetime'] if that's the name

# now extract hour
df['hour'] = df['date'].dt.hour
df['sin_h'] = np.sin(2 * np.pi * df['hour'] / 24)
df['cos_h'] = np.cos(2 * np.pi * df['hour'] / 24)

In [13]:
df = df.dropna()

In [14]:
# 3. train/test split (last 720 rows locked)
train_end = len(df) - 720
test_df = df.iloc[train_end:].copy()

FEATS = [c for c in df.columns if '_L' in c] + ['sin_h', 'cos_h']


In [17]:
preds = []
for t in range(train_end, len(df)):

    # Rolling window
    window = df.iloc[t - 168:t]
    X = window[FEATS].values
    y = window["wind_speed"].values

    # Grid search (NO OOB for DecisionTree!)
    best_mae, best_model = np.inf, None
    for d in [4, 6, 8, 10]:
        for ms in [5, 10, 20]:

            mdl = DecisionTreeRegressor(
                max_depth=d,
                min_samples_leaf=ms,
                random_state=42
            )
            mdl.fit(X, y)

            # use in-sample MAE (since no OOB for DecisionTree)
            mae = np.mean(np.abs(y - mdl.predict(X)))

            if mae < best_mae:
                best_mae = mae
                best_model = mdl

    # predict next step
    x_next = df.iloc[t][FEATS].values.reshape(1, -1)
    preds.append(best_model.predict(x_next)[0])


In [21]:
from sklearn import metrics as m
import numpy as np

test_df['pred_dt2'] = preds

# MAE
mae = m.mean_absolute_error(test_df['wind_speed'], test_df['pred_dt2'])

# RMSE (manual)
mse = m.mean_squared_error(test_df['wind_speed'], test_df['pred_dt2'])
rmse = np.sqrt(mse)

# Safe MAPE
actual = test_df['wind_speed'].values
pred   = test_df['pred_dt2'].values

nonzero_mask = actual != 0
if nonzero_mask.sum() == 0:
    mape = np.inf
else:
    mape = np.mean(np.abs((actual[nonzero_mask] - pred[nonzero_mask]) / actual[nonzero_mask])) * 100

print("MAE :", round(mae, 4))
print("RMSE:", round(rmse, 4))
print("MAPE:", round(mape, 2) if np.isfinite(mape) else "inf")


MAE : 0.8824
RMSE: 1.2466
MAPE: 144.27
